In [2]:
# Upload ipynb
from google.colab import files
uploaded = files.upload()

Saving summary_kw_clob2.csv to summary_kw_clob2.csv


In [3]:
# -*- coding: utf-8 -*-
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import rc
import platform

# 한글 폰트 설정
if platform.system() == 'Windows':
    fontprop = 'Malgun Gothic'
    rc('font', family=fontprop)  # Windows의 경우
elif platform.system() == 'Darwin':  # macOS
    rc('font', family='AppleGothic')
else:  # Linux
    rc('font', family='NanumGothic')


import pandas as pd
import numpy as np

f0='New_Clob.csv'; f3='summary_kw_clob2.csv'
df0 = pd.read_csv(f0); df3 = pd.read_csv(f3)
#print(df0.index, df3.index)  # Print the indices of both DataFrames


merged_df = df0.join(df3, how='inner')
#print(merged_df)

filtered_df = merged_df[merged_df['news_importance'] >= 4]
#print(filtered_df.keywords, filtered_df.CONTENTS, filtered_df.summary, filtered_df.keyword_reason, filtered_df.importance_reason)  # Print the filtered DataFrame
#print(filtered_df)

In [4]:
!pip install rank_bm25 squarify --quiet --upgrade

In [5]:
from rank_bm25 import BM25Okapi

# 문서 데이터
#documents = filtered_df['CONTENTS'].tolist()
documents = merged_df['CONTENTS'].tolist()

# 문서를 토큰화
def tokenizer(sent):
  return sent.split(" ")
tokenized_documents = [tokenizer(str(doc)) for doc in documents]

# BM25 모델 생성
bm25 = BM25Okapi(tokenized_documents)

# 쿼리
query = "철광석 공급망"
tokenized_query = tokenizer(query)

# 각 문서와 쿼리 간의 BM25 점수 계산
scores = bm25.get_scores(tokenized_query)

'''
# 결과 출력
for i, score in enumerate(scores):
    print(f"Document {i + 1}: BM25 Score = {score:.4f}")
'''

print(bm25.get_top_n(tokenized_query, documents, n=2))

# BM25 점수를 merged_df에 추가
merged_df['bm25_score'] = scores
# BM25 점수를 기준으로 내림차순 정렬
sorted_df = merged_df.sort_values(by='bm25_score', ascending=False)






['호주의 세계 4위 철광석 생산업체 Fortescue가 2024년 하반기 철광석 출하 실적에서 역대 최대 기록을 달성했다. Iron Bridge 프로젝트의 조업 중단과 Pilbara 지역 폭우에도 불구하고 철광석 출하 차질을 최소화하는 데 성공한 결과다.마이닝닷컴에 따르면 Fortescue는 2024년 7월부터 12월까지 철광석 출하량이 9,710만 톤에 달해 반기 기준 역대 최고 실적을 달성했다고 밝혔다. 특히 10월부터 12월 출하량은 4,940만 톤으로, 전년 동기 4,870만 톤을 소폭 웃돌았다. 이는 악재 속에서도 안정적인 공급망 운영을 이어간 성과로 풀이된다.동사는 이번 실적 발표에서 2025 회계연도 철광석 출하량 전망치를 1억 9,000만 톤에서 최대 2억 톤으로 유지한다고 밝혔다. Iron Bridge 프로젝트가 연내 전면 가동될 것으로 예상됨에 따라 철광석 증산이 기대되며, Fortescue의 공급 역량 강화에 긍정적 영향을 미칠 전망이다.한편, 1월 22일 기준 중국 칭다오항에서 62% Fe 분광 철광석 가격은 톤당 103.61달러로 전일 대비 1.1% 하락했다. 글로벌 수요와 공급 요인이 복합적으로 작용하는 가운데, 향후 철광석 가격 변동성이 주목된다.', '중동 지역 대표 철강·건축 자재 생산업체인 Emsteel이 2020~2024년 5년간 혁신 및 기술 개발 투자를 127% 확대했다고 밝혔다. Emsteel 그룹의 사이드 그무란 알 레메이티(Saeed Ghumran Al Remeithi) 최고경영자(CEO)는 걸프비즈니스(Gulf Business)와의 인터뷰에서 이같이 밝혔다.Emsteel은 아부다비 제조업의 10%, 아랍에미리트(UAE) 철강 시장의 60%를 점유하고 있으며, 70개국 이상에 제품을 수출하고 있다. 알 레메이티 CEO는 혁신을 시장 교란, 글로벌 공급망 차질, 규제 환경 변화에 대응할 수 있는 전략적 차별화 수단으로 보고 있다고 강조했다.그는 디지털 전환과 첨단 제조기술 도입을 주요 성장동력으로 꼽았다. 핵심 기술 

In [ ]:
import squarify

# BM25 점수를 기준으로 상위 10개의 키워드와 점수를 딕셔너리로 생성
top_10 = sorted_df.nlargest(10, 'bm25_score')
top_10['keywords'] = top_10['keywords'].apply(lambda x: ','.join([x.split(',')[i] + ('\n' if (i + 1) % 3 == 0 else '') for i in range(len(x.split(',')))]))
#top_10['keywords'] = top_10['keywords'].apply(lambda x: '\n'.join([x[i:i+20] for i in range(0, len(x), 20)]))  # 20자 단위로 줄바꿈
sorted_twos = list(zip(top_10['keywords'], top_10['bm25_score']))
sorted_dict = dict(sorted_twos)

plt.figure(figsize=(12, 10))

# 빈도값을 0~1까지 값을 갖도록 min-max 표준화
norm = mpl.colors.Normalize(vmin=min(sorted_dict.values()),
                            vmax=max(sorted_dict.values()))

# 정규화된 값을 matplotlib의 cm(color map)에서 Blue 에 적용
colors = [mpl.cm.Blues(norm(value)) for value in sorted_dict.values()]

# squarify 플랏생성(label 단어, 박스 크기는 빈도, 색깔르 cm.Blue, 투명도는 30%
squarify.plot(label=sorted_dict.keys(),
              sizes=sorted_dict.values(),
              color=colors,
              alpha=.7)
#plt.show()


In [12]:
sorted_df.describe()

,CREATION_DATE,SENTIMENTAL,IMPORTANCE,news_importance,bm25_score
count,3.800000e+02,380.000000,380.000000,380.000000,380.000000
mean,2.024589e+07,-0.126316,3.350000,3.528947,0.629175
std,4.803774e+03,0.995951,0.860217,0.535499,1.243376
min,2.024010e+07,-5.000000,1.000000,1.000000,0.000000
25%,2.024071e+07,0.000000,3.000000,3.000000,0.000000
50%,2.025011e+07,0.000000,3.000000,4.000000,0.000000
75%,2.025022e+07,0.000000,4.000000,4.000000,1.105271
max,2.025043e+07,5.000000,5.000000,5.000000,7.528504


In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-vpszykz5/kobert-tokenizer_621151b3fc224240815b69446b019c41
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-vpszykz5/kobert-tokenizer_621151b3fc224240815b69446b019c41
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 5c46b1c68e4755b54879431bd302db621f4d2f47
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=982cf996864154fc7b78fccff67a2e5c85a9238acb6a3d68937d3a28c381f3bb
  Stored in directory: /tmp/pip-ephem-wheel-cache-_xgh8pcx/wheels/b7/95/7c/3f38b039ae9422cc4a51702b37e352d0585442cbe60285f1f2
Successfully built kobert_tokenizer


In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1")
model = AutoModel.from_pretrained("skt/kobert-base-v1")
sentences = merged_df['summary'].tolist()
embeddings = tokenizer.tokenize(sentences)

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [ ]:
from sentence_transformers import SentenceTransformer, models
#from kobert_transformers import get_tokenizer
#tokenizer = get_tokenizer()

#pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
from kobert_tokenizer import KoBERTTokenizer
# KoBERT 모델과 토크나이저 로드
model_name = "skt/kobert-base-v1"
tokenizer = KoBERTTokenizer.from_pretrained(model_name)

# SentenceTransformer 모델 구성
#word_embedding_model = models.Transformer(model_name, max_seq_length=256)
#pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode_mean_tokens=True)

# SentenceTransformer 모델 생성
#model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

# 문장 리스트
sentences = merged_df['summary'].tolist()
print(len(sentences), len(sentences[0]), sentences[0][:127])

# 문장 임베딩 생성
inputs = tokenizer(sentences)
len(inputs)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


380 191 포트 헤들랜드에서 철광석 운송을 담당하는 호주 포트레스큐 메탈스 그룹(FMG)의 철도 운영이 1월 3일까지 정상화될 것으로 예상됩니다. 12월 30일 발생한 열차 탈선 사고로 인한 영향은 없으며, 충분한 재고로 인해 수출량에 미


3

In [28]:
# https://github.com/jhgan00/ko-sentence-transformers
# https://pypi.org/project/ko-sentence-transformers/

from sentence_transformers import SentenceTransformer, util
import numpy as np

#model_name = "snunlp/KR-SBERT-V40K-klueNLI-augSTS"
model_name = "jhgan/ko-sroberta-multitask"
#model_name = "monologg/kobert"
#model_name = "jhgan/ko-sbert-sts"
#model_name = "distiluse-base-multilingual-cased-v1"

embedder = SentenceTransformer(model_name, trust_remote_code=True)


In [266]:
sentences = '국가유산청이 한반도 고대 제철기술을 재현하는 행사를 추진한다. 당국은 이번 행사를 통해 중원지역 고대 제철기술도 표준화하겠단 계획이다. 국가유산청 국립문화유산연구원 국립중원문화유산연구소는 오는 4월 24일, 충북 청주시 중원연구소 제철기술 복원실험장에서 ‘2025년도 고대 제철기술 복원(제련)실험’을 진행한다고 밝혔다. 제련(製鍊)은 철광석을 녹여 철을 만드는 생산과정을 뜻한다. 이번 복원실험은 중원지역 고대 철 생산기술의 실체를 밝히고 이를 기반으로 한 표준실험지침(매뉴얼)을 마련하기 위해 진행된다. 이에 따라 당일, ‘제련실험’과 ‘사용실험’이 각각 진행될 예정이다. 중원연구소는 지난 2014년부터 진천 석장리, 충주 칠금동 등 삼국시대 제철유적에서 확인된 제련로를 토대로 송풍 방식, 조업 조건 등 고대 제철기술을 고증·재현해왔다. 이번 16차 제련실험은 생산된 철과 슬래그 등 제련 부산물의 과학적 분석 결과가 제철유적 출토 유물과 가장 유사한 특성을 보였던 15차 실험(‘24년 실시)을 통해 복원된 고대 철 제련기술의 신뢰성과 반복 재현 가능성을 검증하기 위해 동일한 환경에서 진행될 예정이다. 국가유산청은 실험을 통해 수집되는 온도 변화, 생산된 철의 양, 부산물의 구성 등 과학적 자료는 향후 고대 철 생산기술 표준실험지침(매뉴얼) 수립을 위한 핵심 자료로 활용할 예정이다. 이번 실험이 성공적으로 마무리될 경우, 중원지역 고대 제철기술에 대한 정확한 공정 기준이 마련되어 향후 고대 철기문화 연구와 전통 철기 복원 분야에서 획기적인 진전을 가져올 것으로 기대된다. 24일 진행되는 제련실험은 별도의 사전 신청 없이 현장 접수로 참가할 수 있으며, 참가자들은 철 생산과정을 학예연구사의 전문적인 해설과 함께 관찰할 수 있다. 한편, 국가유산청은 올해 하반기엔 고대 제련 과정을 통한 ‘사용실험’을 진행하여 복원한 고대 철제 무기의 실제 활용성과 성능을 검증할 계획이다. 2024년 15차 제련실험 장면 (국가유산청)'
corpus_embeddings = embedder.encode(sentences, convert_to_tensor=True, show_progress_bar=True)

queries = '국가유산청이 중원지역 고대 제철기술을 재현하는 행사를 추진하며, 이를 통해 고대 철 생산기술의 표준화된 실험지침을 마련할 계획이다. 이번 실험은 고대 철 제련기술의 신뢰성과 반복 재현 가능성을 검증하기 위해 진행되며, 성공 시 고대 철기문화 연구와 전통 철기 복원 분야에서 획기적인 진전을 기대하고 있다. 이 뉴스는 고대 제철기술의 복원과 관련된 내용으로, 현재 철강 산업의 원자재 시장에 직접적인 영향을 미치지는 않지만, 전통 기술 연구와 복원 분야에 일부 영향을 미칠 수 있다.'
query_embeddings = embedder.encode(queries, convert_to_tensor=True, show_progress_bar=True)

util.pytorch_cos_sim(query_embeddings, corpus_embeddings)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

IndexError: index out of range in self

In [264]:
rouge.get_scores(' '.join(okt.nouns(queries)), ' '.join(okt.nouns(sentences)), avg=True)

{'rouge-1': {'r': 0.30708661417322836,
  'p': 0.7647058823529411,
  'f': 0.4382022431025124},
 'rouge-2': {'r': 0.17733990147783252,
  'p': 0.5538461538461539,
  'f': 0.2686567127436512},
 'rouge-l': {'r': 0.25984251968503935,
  'p': 0.6470588235294118,
  'f': 0.3707865127654336}}

In [167]:
from konlpy.tag import Okt
okt = Okt() # 객체 생성

#okt.pos(sentences, norm=True, stem=True)  # 어떤 형태소 종류인지 튜플로 반환
' '.join(okt.nouns(sentences))

'중국 철 강재 수출 톤 기록 중국 해관 총서 발표 이 누적 수출 톤 년 동기 대비 증가 철 강재 수입 톤 기록 누적 수입량 톤 년 동기 대비 감소 철광석 및 정광 수입 톤 기록 누적 수입량 톤 년 동기 대비 감소 석탄 및 갈탄 수입량 톤 누적 수입 톤 년 기간'

In [174]:
from rouge import Rouge

rouge = Rouge()
rouge.get_scores(' '.join(okt.nouns(queries)), ' '.join(okt.nouns(sentences)), avg=True)
#rouge.get_scores(' '.join(okt.morphs(queries)), ' '.join(okt.morphs(sentences)), avg=True)

{'rouge-1': {'r': 0.3333333333333333,
  'p': 0.9473684210526315,
  'f': 0.49315068108087823},
 'rouge-2': {'r': 0.1342281879194631,
  'p': 0.4878048780487805,
  'f': 0.21052631240498618},
 'rouge-l': {'r': 0.28703703703703703,
  'p': 0.8157894736842105,
  'f': 0.4246575303959467}}

In [155]:
from rouge_score import rouge_scorer

# Initialize a scorer for multiple ROUGE variants
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Calculate scores
scores = scorer.score(sentences, queries)
scores

{'rouge1': Score(precision=1.0, recall=0.10638297872340426, fmeasure=0.1923076923076923),
 'rouge2': Score(precision=1.0, recall=0.08695652173913043, fmeasure=0.16),
 'rougeL': Score(precision=1.0, recall=0.10638297872340426, fmeasure=0.1923076923076923)}

In [161]:
!pip install rouge_score rouge evaluate KoNLPy --upgrade --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 21.0 MB/s eta 0:00:00


In [86]:
from rouge_score import rouge_scorer

In [88]:
l = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
l.score(sentences, queries)

{'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0),
 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0),
 'rougeL': Score(precision=0, recall=0, fmeasure=0)}

In [29]:
sentences = merged_df['summary'].tolist()
corpus_embeddings = embedder.encode(sentences, convert_to_tensor=True, show_progress_bar=True)

queries = merged_df['CONTENTS'].tolist()
query_embeddings = embedder.encode(queries, convert_to_tensor=True, show_progress_bar=True)


Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

In [30]:
list_scores = []
for i in range(len(sentences)):
  cos_scores = util.pytorch_cos_sim(query_embeddings[i], corpus_embeddings[i])#[0]
  #print(cos_scores)
  cos_scores = cos_scores.cpu()
  list_scores.append(cos_scores)
np.mean(list_scores)

np.float32(0.66564524)

In [30]:
list_scores = []
for i in range(len(sentences)):
  cos_scores = util.pytorch_cos_sim(query_embeddings[i], corpus_embeddings[i])#[0]
  #print(cos_scores)
  cos_scores = cos_scores.cpu()
  list_scores.append(cos_scores)
np.mean(list_scores)

np.float32(0.66564524)

In [21]:
# https://github.com/jhgan00/ko-sentence-transformers
# https://pypi.org/project/ko-sentence-transformers/

from sentence_transformers import SentenceTransformer, util
import numpy as np

model_name = "jhgan/ko-sroberta-multitask"
#model_name = "monologg/kobert"
#model_name = "jhgan/ko-sbert-sts"
embedder = SentenceTransformer(model_name)

#sentences = merged_df['summary'].tolist()
#sentences = merged_df['CONTENTS'].tolist()
corpus_embeddings = embedder.encode(documents, convert_to_tensor=True, show_progress_bar=True)

# Query sentences:
#queries = ['니켈', '공급망']
queries = [query]
queries = ['철광석', '공급망']

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 10
i=0
for q in queries:
 i = i+1
 query_embedding = embedder.encode(q, convert_to_tensor=True)
 cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
 cos_scores = cos_scores.cpu()
 if i==1:
  merged_df['bert_score1']=cos_scores
 elif i==2:
  merged_df['bert_score2']=cos_scores

 #We use np.argpartition, to only partially sort the top_k results
 top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

 print("\n\n======================\n\n")
 print("Query:", q)
 print("\nTop 5 most similar sentences in corpus:")

 for idx in top_results[0:top_k]:
  print(documents[idx].strip(), "(Score: %.4f)" % (cos_scores[idx]))

Batches:   0%|          | 0/12 [00:00<?, ?it/s]





Query: 철광석

Top 5 most similar sentences in corpus:
우드맥킨지의 철광석? 2024년 5가지 볼거리 보고서 표지. (출처우드맥킨지) 시장조사기관 우드맥킨지(WoodMckenzie)는 최근 철광석? 2024년 5가지 볼거리(Iron ore? 5 things to look for in 2024)라는 보고서를 발표하고, 세계 최대 철강 생산국인 중국의 경기 부진 및 정책 변동성, 고품위 철광석 수요 급증에 따른 광산 개발 복잡성 증가, 아프리카의 시만두(Simandou) 프로젝트, 중국의 자체 생산 확대 지연, 철강산업의 탈탄소화에 따른 새로운 녹색 허브 탄생 등 5가지 주요 이슈를 제기했다. 2024년 중국 경제, 인프라 및 제조업 중심 개선에도 부동산 부진에 철강 수요 1% 미만 증가 첫째, 보고서는 세계 최대의 철강 생산국인 중국이 올해 일정 수준의 경기 회복에도 부동산 부문의 침체가 지속되며 철강 수요는 크게 늘지 않을 것으로 예상했다. 2023년은 중국 경제에 유망한 한 해가 될 예정이었다. 이는 제로코로나 정책의 폐지를 의미하며 봉쇄 규제 완화와 산업 활동의 강력한 반등을 기대하게 했다. 그러나 현실은 매우 달라 보였는데, 중국의 둔화로 인해 철강 시장이 연중 침체됐다. 국내적으로는 부동산 경기 침체, 지방정부 부채 증가, 낮은 시장 신뢰도가 중국의 경제 성장을 위협하고 있으며, 대외적으로는 지정학적 리스크와 공급망의 재배치로 인해 고부가가치 부문을 개발하고 품질 중심의 개발로 전환하려는 중국의 야망이 꺾였다. 그럼에도 불구하고 중국의 조강 생산량은 2023년에 10억 톤을 약간 넘는 수준으로 증가했으며, 기계, 자동차 및 인프라의 수출 및 수요 증가로 어려움을 겪고 있는 부동산 부문의 손실을 부분적으로 상쇄했다. 보고서는 중국의 2024년 부동산 시장 전망에 대해 경기가 이미 바닥에 이르렀고, 서서히 반등할 것이라는 전망을 내놓았다. 우드맥킨지는 우리는 베이징이 더 많은 부동산 개발업체들에게 신용 유동성을 

In [22]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

scaler1 = MinMaxScaler(feature_range=(0, .5))
scaler2 = MinMaxScaler(feature_range=(0, 1))

# min-max scaling 수행
#norm1 = scaler1.fit_transform(merged_df['bm25_score'].values.reshape(-1, 1))
#norm2 = scaler2.fit_transform(merged_df['bert_score'].values.reshape(-1, 1))
norm3 = scaler2.fit_transform(merged_df['bert_score1'].values.reshape(-1, 1))
norm4 = scaler2.fit_transform(merged_df['bert_score2'].values.reshape(-1, 1))

In [23]:
#merged_df['bm25_norm']=norm1
#merged_df['bert_norm']=norm2
merged_df['bert_norm1']=norm3
merged_df['bert_norm2']=norm4

In [24]:
merged_df['mean_score'] = merged_df[['bm25_norm', 'bert_norm']].mean(axis=1)

In [25]:
sorted_df = merged_df.sort_values(by='mean_score', ascending=False)
sorted_df.to_csv('sorted_df2.csv', index=False)
sorted_df.to_excel('sorted_df2.xlsx', index=False)

In [ ]:
import squarify

# BM25 점수를 기준으로 상위 10개의 키워드와 점수를 딕셔너리로 생성
top_10 = sorted_df.nlargest(10, 'bm25_score')
top_10['keywords'] = top_10['keywords'].apply(lambda x: ','.join([x.split(',')[i] + ('\n' if (i + 1) % 3 == 0 else '') for i in range(len(x.split(',')))]))
#top_10['keywords'] = top_10['keywords'].apply(lambda x: '\n'.join([x[i:i+20] for i in range(0, len(x), 20)]))  # 20자 단위로 줄바꿈
sorted_twos = list(zip(top_10['keywords'], top_10['bm25_score']))
sorted_dict = dict(sorted_twos)

plt.figure(figsize=(12, 10))

# 빈도값을 0~1까지 값을 갖도록 min-max 표준화
norm = mpl.colors.Normalize(vmin=min(sorted_dict.values()),
                            vmax=max(sorted_dict.values()))

# 정규화된 값을 matplotlib의 cm(color map)에서 Blue 에 적용
colors = [mpl.cm.Blues(norm(value)) for value in sorted_dict.values()]

# squarify 플랏생성(label 단어, 박스 크기는 빈도, 색깔르 cm.Blue, 투명도는 30%
squarify.plot(label=sorted_dict.keys(),
              sizes=sorted_dict.values(),
              color=colors,
              alpha=.7)
#plt.show()

In [ ]:
!pip install feedparser goose3 --upgrade --quiet

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 22.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.4/113.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 8.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import feedparser
import requests
from goose3 import Goose
from goose3.text import StopWordsKorean

In [ ]:
def get_data(url):
    try:
        res = requests.get(url)
        html = res.text
        data = feedparser.parse(html)
        print(data.feed.title)
        return data
    except:
        return None

url = 'https://www.yna.co.kr/rss/industry.xml'
url = 'https://www.chosun.com/arc/outboundfeeds/rss/category/economy/?outputType=xml'
parsed_data = get_data(url)

num_entries = 2 # article 정보 확인을 위해 임의로 번호를 지정했습니다.
article = parsed_data['entries'][num_entries]
article

조선일보


{'title': '31평 아파트 낙찰받자 선도지구 선정…1년도 안돼 9천만원 벌었다',
 'title_detail': {'type': 'text/plain',
  'language': None,
  'base': '',
  'value': '31평 아파트 낙찰받자 선도지구 선정…1년도 안돼 9천만원 벌었다'},
 'links': [{'rel': 'alternate',
   'type': 'text/html',
   'href': 'https://www.chosun.com/economy/realty/investment_trends/2025/05/13/KTVP5NXI3EKBWQ7IFA2I2CFS7A/'}],
 'link': 'https://www.chosun.com/economy/realty/investment_trends/2025/05/13/KTVP5NXI3EKBWQ7IFA2I2CFS7A/',
 'id': 'https://www.chosun.com/economy/realty/investment_trends/2025/05/13/KTVP5NXI3EKBWQ7IFA2I2CFS7A/',
 'guidislink': False,
 'authors': [{'name': '이승우 기자(땅집고)'}],
 'author': '이승우 기자(땅집고)',
 'author_detail': {'name': '이승우 기자(땅집고)'},
 'summary': '[땅집고] 재건축 아파트 경매에 관심이 많았던 A씨. 지난해 수도권 1기 신도시인 경기도 군포시 산본신도시 한 아파트 매물을 주목했다. 지상 20층, 822가구로 초등학교를 품고 있는 이른바 ‘초품아’였다. 1993년 준공해 30년이 넘었지만 산본 중심에 있어 기본적인 인프라는 좋았다. 무엇보다 노후계획도시 특별법에 따른 재건축을 기대할 수 있는 곳이었다.',
 'summary_detail': {'type': 'text/html',
  'language': None,
  'base': '',
  'value': '[땅집고] 재건축 아파

In [ ]:
article['content']['value']

TypeError: list indices must be integers or slices, not str

In [ ]:
reader = Goose({'stopwords_class':StopWordsKorean})

In [ ]:
url = 'https://news.chosun.com/site/data/html_dir/2020/02/13/2020021301270.html'
contents = reader.extract(url=url)
contents

In [ ]:
print(contents.cleaned_text)